##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Keras Tuner 简介

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/tutorials/keras/keras_tuner"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png"> 在 TensorFlow.org 上查看</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/keras/keras_tuner.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/keras/keras_tuner.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/keras/keras_tuner.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table>

## 概述

Keras Tuner 是一个库，可帮助您为 TensorFlow 程序选择最佳的超参数集。为您的机器学习 (ML) 应用选择正确的超参数集，这一过程称为*超参数调节*或*超调*。

超参数是控制训练过程和 ML 模型拓扑的变量。这些变量在训练过程中保持不变，并会直接影响 ML 程序的性能。超参数有两种类型：

1. **模型超参数**：影响模型的选择，例如隐藏层的数量和宽度
2. **算法超参数**：影响学习算法的速度和质量，例如随机梯度下降 (SGD) 的学习率以及 k 近邻 (KNN) 分类器的近邻数

在本教程中，您将使用 Keras Tuner 对图像分类应用执行超调。

## 设置

In [2]:
import tensorflow as tf
from tensorflow import keras

安装并导入 Keras Tuner。

In [3]:
!pip install -q -U keras-tuner

You should consider upgrading via the '/tmpfs/src/tf_docs_env/bin/python -m pip install --upgrade pip' command.


In [4]:
import keras_tuner as kt

## 下载并准备数据集

在本教程中，您将使用 Keras Tuner 为某个对 [Fashion MNIST 数据集](https://github.com/zalandoresearch/fashion-mnist)内的服装图像进行分类的机器学习模型找到最佳超参数。

加载数据。

In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [6]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## 定义模型

构建用于超调的模型时，除了模型架构之外，还要定义超参数搜索空间。您为超调设置的模型称为*超模型*。

您可以通过两种方式定义超模型：

- 使用模型构建工具函数
- 将 Keras Tuner API 的 `HyperModel` 类子类化

您还可以将两个预定义的 `HyperModel` 类（[HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class) 和 [HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class)）用于计算机视觉应用。

在本教程中，您将使用模型构建工具函数来定义图像分类模型。模型构建工具函数将返回已编译的模型，并使用您以内嵌方式定义的超参数对模型进行超调。

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## 实例化调节器并执行超调

实例化调节器以执行超调。Keras Tuner 提供了四种调节器：`RandomSearch`、`Hyperband`、`BayesianOptimization` 和 `Sklearn`。在本教程中，您将使用 [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) 调节器。

要实例化 Hyperband 调节器，必须指定超模型、要优化的 `objective` 和要训练的最大周期数 (`max_epochs`)。

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

2021-08-13 23:32:45.391821: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


2021-08-13 23:32:45.398545: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-13 23:32:45.399450: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-13 23:32:45.401464: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-13 23:32:45.401961: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

Hyperband 调节算法使用自适应资源分配和早停法来快速收敛到高性能模型。该过程采用了体育竞技争冠模式的排除法。算法会将大量模型训练多个周期，并仅将性能最高的一半模型送入下一轮训练。Hyperband 通过计算 1 + log<sub><code>factor</code></sub>(`max_epochs`) 并将其向上舍入到最接近的整数来确定要训练的模型的数量。

创建回调以在验证损失达到特定值后提前停止训练。

In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

运行超参数搜索。除了上面的回调外，搜索方法的参数也与 `tf.keras.model.fit` 所用参数相同。

In [10]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 28s]
val_accuracy: 0.8572499752044678

Best val_accuracy So Far: 0.8910833597183228
Total elapsed time: 00h 05m 57s
INFO:tensorflow:Oracle triggered exit



The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 192 and the optimal learning rate for the optimizer
is 0.001.



## 训练模型

使用从搜索中获得的超参数找到训练模型的最佳周期数。

In [11]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50


   1/1500 [..............................] - ETA: 5:33 - loss: 2.2131 - accuracy: 0.2812

  33/1500 [..............................] - ETA: 2s - loss: 1.2470 - accuracy: 0.5729  

  66/1500 [>.............................] - ETA: 2s - loss: 1.0216 - accuracy: 0.6491

 100/1500 [=>............................] - ETA: 2s - loss: 0.9376 - accuracy: 0.6759

...

1423/1500 [===========================>..] - ETA: 0s - loss: 0.0825 - accuracy: 0.9693

1458/1500 [============================>.] - ETA: 0s - loss: 0.0825 - accuracy: 0.9692

1490/1500 [============================>.] - ETA: 0s - loss: 0.0827 - accuracy: 0.9692

1500/1500 [==============================] - 3s 2ms/step - loss: 0.0828 - accuracy: 0.9691 - val_loss: 0.5169 - val_accuracy: 0.8861


Best epoch: 25


重新实例化超模型并使用上面的最佳周期数对其进行训练。

In [12]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/25


   1/1500 [..............................] - ETA: 5:34 - loss: 2.3077 - accuracy: 0.2812

  33/1500 [..............................] - ETA: 2s - loss: 1.2813 - accuracy: 0.5644  

  68/1500 [>.............................] - ETA: 2s - loss: 1.0193 - accuracy: 0.6507

 102/1500 [=>............................] - ETA: 2s - loss: 0.9092 - accuracy: 0.6869

...

1394/1500 [==========================>...] - ETA: 0s - loss: 0.1418 - accuracy: 0.9467

1428/1500 [===========================>..] - ETA: 0s - loss: 0.1425 - accuracy: 0.9465

1462/1500 [============================>.] - ETA: 0s - loss: 0.1428 - accuracy: 0.9466

1496/1500 [============================>.] - ETA: 0s - loss: 0.1428 - accuracy: 0.9466

1500/1500 [==============================] - 3s 2ms/step - loss: 0.1430 - accuracy: 0.9466 - val_loss: 0.3506 - val_accuracy: 0.8925


要完成本教程，请在测试数据上评估超模型。

In [13]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

  1/313 [..............................] - ETA: 26s - loss: 0.1582 - accuracy: 0.9688

 28/313 [=>............................] - ETA: 0s - loss: 0.3474 - accuracy: 0.8951 

 55/313 [====>.........................] - ETA: 0s - loss: 0.3525 - accuracy: 0.8938

 83/313 [======>.......................] - ETA: 0s - loss: 0.3579 - accuracy: 0.8908

111/313 [=========>....................] - ETA: 0s - loss: 0.3976 - accuracy: 0.8843

...

248/313 [======================>.......] - ETA: 0s - loss: 0.4007 - accuracy: 0.8872

275/313 [=========================>....] - ETA: 0s - loss: 0.3994 - accuracy: 0.8873

303/313 [============================>.] - ETA: 0s - loss: 0.3960 - accuracy: 0.8886

313/313 [==============================] - 1s 2ms/step - loss: 0.3943 - accuracy: 0.8890


[test loss, test accuracy]: [0.3942869007587433, 0.8889999985694885]


`my_dir/intro_to_kt` 目录中包含了在超参数搜索期间每次试验（模型配置）运行的详细日志和检查点。如果重新运行超参数搜索，Keras Tuner 将使用这些日志中记录的现有状态来继续搜索。要停用此行为，请在实例化调节器时传递一个附加的 `overwrite = True` 参数。

## 总结

在本教程中，您学习了如何使用 Keras Tuner 调节模型的超参数。要详细了解 Keras Tuner，请查看以下其他资源：

- [TensorFlow 博客上的 Keras Tuner](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
- [Keras Tuner 网站](https://keras-team.github.io/keras-tuner/)

另请查看 TensorBoard 中的 [HParams Dashboard](https://tensorflow.google.cn/tensorboard/hyperparameter_tuning_with_hparams)，以交互方式调节模型超参数。